In [38]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

from cardinality_estimation.featurizer import Featurizer
from query_representation.query import load_qrep

import glob
import random
import os
import json
import time
from collections import defaultdict

from IPython.display import HTML
from IPython.display import display_html

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
#TRAINDIR = os.path.join(os.path.join("", "queries"), "imdb")
#TRAINDIR = os.path.join(os.path.join("", "queries"), "tpcds")
TRAINDIR = os.path.join(os.path.join("", "queries"), "job")
print(TRAINDIR)

queries/job


In [55]:

def load_qdata(fns):
    qreps = []
    for qfn in fns:
        qrep = load_qrep(qfn)
        # TODO: can do checks like no queries with zero cardinalities etc.
        qreps.append(qrep)
        template_name = os.path.basename(os.path.dirname(qfn))
        qrep["name"] = os.path.basename(qfn)
        qrep["template_name"] = template_name
    return qreps

def get_query_fns(basedir, template_fraction=1.0):
    fns = []
    tmpnames = list(glob.glob(os.path.join(basedir, "*")))
    assert template_fraction <= 1.0
    print(tmpnames)
    for qi,qdir in enumerate(tmpnames):
        if os.path.isfile(qdir):
            continue
        print(qdir)
        template_name = os.path.basename(qdir)
        # let's first select all the qfns we are going to load
        qfns = list(glob.glob(os.path.join(qdir, "*.pkl")))
        qfns.sort()
        num_samples = max(int(len(qfns)*template_fraction), 1)
        random.seed(1234)
        qfns = random.sample(qfns, num_samples)
        fns += qfns
    return fns

In [56]:
train_qfns = get_query_fns(TRAINDIR, template_fraction = 1.0)
trainqs = load_qdata(train_qfns)

['queries/job/all_job']
queries/job/all_job


In [57]:
print(len(trainqs))

113


In [58]:
allconstants = defaultdict(set)
constantmaxs = defaultdict(int)

In [64]:
for query in trainqs:
    for node in query["join_graph"].nodes():
        if not "pred_cols" in query["join_graph"].nodes()[node]:
            continue
        for ci, col in enumerate(query["join_graph"].nodes()[node]["pred_cols"]):
            consts = query["join_graph"].nodes()[node]["pred_vals"][ci]
            #print(consts)
            if isinstance(consts, dict):
                consts = consts["literal"]
            elif not isinstance(consts, list):
                consts = [consts]
            
            for const in consts:
                if isinstance(const, dict):
                    const = const["literal"]
                allconstants[col].add(const)
            if constantmaxs[col] < len(consts):
                constantmaxs[col] = len(consts)
#             if len(consts) > 20:
#                 print(consts)
#                 print(query["sql"])
#                 print(query["name"])

In [65]:
for k,v in allconstants.items():
    print(k, len(v), constantmaxs[k])

n.name 12 1
t.production_year 16 2
k.keyword 52 25
n.gender 3 1
t.title 25 7
rt.role 16 16
it.info 23 14
ci.note 30 24
cn.country_code 13 4
cn.name 24 20
mi.info 34 10
it1.info 16 13
kt.kind 11 5
it2.info 19 14
mi_idx.info 9 2
ct.kind 16 20
pi.note 11 12
n.name_pcode_cf 3 2
lt.link 17 8
an.name 2 1
mc.note 16 1
kt2.kind 2 2
t2.production_year 5 2
mi_idx2.info 3 2
cn1.country_code 6 4
kt1.kind 2 2
cct1.kind 18 17
chn.name 9 5
cct2.kind 15 17
mi.note 2 1
t.episode_nr 4 2
it3.info 8 6
n1.name 2 1


In [67]:
data = defaultdict(list)
for query in trainqs:
    jg = query["join_graph"]
    sg = query["subset_graph"]
    
    for node in jg.nodes():
        #data["num_unique_cols"].append(len)
        #print(query["sql"])
        #print(query[""])
        #print(query["join_graph"].nodes()[node])
        if not "pred_cols" in query["join_graph"].nodes()[node]:
            continue
        if len(jg.nodes()[node]["pred_cols"]) == 0:
            continue
        #data["num_unique_cols"].append(jg.nodes()[node]["pred_cols"])
        
        alias_key = tuple([node])
        cards = sg.nodes()[alias_key]["cardinality"]
        #print(cards)
        sel = cards["actual"] / cards["total"]
        sel = min(sel, 1.00)
        curcard = cards["actual"]
        
        seencols = []
        seenops = []
        consts = []
            
#         data["selectivity"].append(sel)
#         data["cardinality"].append(curcard)
        for ci, col in enumerate(query["join_graph"].nodes()[node]["pred_cols"]):
            
            op = jg.nodes()[node]["pred_types"][ci]           
            if op not in seenops:
                seenops.append(op)
            if col not in seencols:
                seencols.append(seencols)
            
            if isinstance(jg.nodes()[node]["pred_vals"][ci], int):
                consts.append(jg.nodes()[node]["pred_vals"][ci])
            else:
                consts += jg.nodes()[node]["pred_vals"][ci]
        
        data["input"].append(jg.nodes()[node]["real_name"])     
        if "like" in seenops:
            data["like_ops"].append(1)
        else:
            data["like_ops"].append(0)

        if "lt" in seenops:
            data["cont_ops"].append(1)
        else:
            data["cont_ops"].append(0)

        if "in" in seenops:
            data["in_ops"].append(1)
        else:
            data["in_ops"].append(0)

        if "in" in seenops or "eq" in seenops:
            data["discrete_ops"].append(1)
            data["num_discrete_consts"].append(len(consts))
        else:
            data["discrete_ops"].append(0)
            data["num_discrete_consts"].append(0)
        
        data["num_ops"].append(len(jg.nodes()[node]["pred_types"]))
        data["num_cols_all"].append(len(jg.nodes()[node]["pred_cols"]))
        data["num_unique_ops"].append(len(seenops))
        data["unique_filter_cols"].append(len(seencols))
        data["equal_dates"].append(0.0)

In [68]:
df = pd.DataFrame(data)

In [69]:
print(df.keys())

Index(['input', 'like_ops', 'cont_ops', 'in_ops', 'discrete_ops',
       'num_discrete_consts', 'num_ops', 'num_cols_all', 'num_unique_ops',
       'unique_filter_cols', 'equal_dates'],
      dtype='object')


In [70]:
HTML(df[["num_ops", "num_unique_ops", "unique_filter_cols",
        "num_discrete_consts"]].\
     describe(percentiles=[0.9,0.99]).reset_index().to_html(index=False))

index,num_ops,num_unique_ops,unique_filter_cols,num_discrete_consts
count,613.000000,613.000000,613.000000,613.000000
mean,1.169657,1.066884,1.169657,1.404568
std,0.465046,0.250025,0.465046,2.020187
min,1.000000,1.000000,1.000000,0.000000
50%,1.000000,1.000000,1.000000,1.000000
90%,2.000000,1.000000,2.000000,4.000000
99%,3.000000,2.000000,3.000000,10.000000
max,4.000000,2.000000,4.000000,10.000000


In [71]:
# HTML(df[["like_ops", "discrete_ops", "cont_ops", "in_ops", "equal_dates"]].\
#      describe(percentiles=[0.9,0.99]).reset_index().to_html(index=False))
HTML(df[["like_ops", "discrete_ops", "cont_ops", "in_ops"]].\
     describe(percentiles=[0.9,0.99]).reset_index().to_html(index=False))

index,like_ops,discrete_ops,cont_ops,in_ops,equal_dates
count,613.000000,613.000000,613.000000,613.000000,613.0
mean,0.192496,0.678630,0.168026,0.197390,0.0
std,0.394582,0.467384,0.374195,0.398354,0.0
min,0.000000,0.000000,0.000000,0.000000,0.0
50%,0.000000,1.000000,0.000000,0.000000,0.0
90%,1.000000,1.000000,1.000000,1.000000,0.0
99%,1.000000,1.000000,1.000000,1.000000,0.0
max,1.000000,1.000000,1.000000,1.000000,0.0


In [51]:
HTML(df[["selectivity", "cardinality"]].\
     describe(percentiles=[0.9,0.99]).reset_index().round(3).to_html(index=False))

KeyError: "None of [Index(['selectivity', 'cardinality'], dtype='object')] are in the [columns]"

In [73]:
#df.groupby("input").count()

,like_ops,cont_ops,in_ops,discrete_ops,num_discrete_consts,num_ops,num_cols_all,num_unique_ops,unique_filter_cols,equal_dates
input,,,,,,,,,,
aka_name,3,3,3,3,3,3,3,3,3,3
cast_info,30,30,30,30,30,30,30,30,30,30
char_name,8,8,8,8,8,8,8,8,8,8
comp_cast_type,37,37,37,37,37,37,37,37,37,37
company_name,58,58,58,58,58,58,58,58,58,58
company_type,22,22,22,22,22,22,22,22,22,22
info_type,96,96,96,96,96,96,96,96,96,96
keyword,69,69,69,69,69,69,69,69,69,69
kind_type,29,29,29,29,29,29,29,29,29,29


In [74]:
df.head(5)

,input,like_ops,cont_ops,in_ops,discrete_ops,num_discrete_consts,num_ops,num_cols_all,num_unique_ops,unique_filter_cols,equal_dates
0,name,1,0,0,0,0,1,1,1,1,0.0
1,title,0,1,0,0,0,1,1,1,1,0.0
2,keyword,0,0,1,1,8,1,1,1,1,0.0
3,name,1,0,0,1,2,2,2,2,2,0.0
4,title,1,1,0,0,0,2,2,2,2,0.0
